In [9]:
import whisper
from moviepy.editor import VideoFileClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
import os
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Setare pentru ImageMagick
os.environ["IMAGEMAGICK_BINARY"] = r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"

# 📍 Configurări inițiale
SUBTITLE_FILE = r"C:\Users\alex_\Downloads\subtitles.srt"
OUTPUT_VIDEO = r"C:\Users\alex_\Downloads\video_with_subtitles.mp4"

# 🔹 1. Funcție pentru generarea fișierului SRT cu timestamp-uri corecte
def save_subtitles(whisper_result, output_srt=SUBTITLE_FILE):
    """ Salvează subtitrările într-un fișier .srt cu timestamp-uri reale """
    with open(output_srt, "w", encoding="utf-8") as f:
        for i, segment in enumerate(whisper_result["segments"]):
            start_time = format_srt_time(segment["start"])
            end_time = format_srt_time(segment["end"])
            text = segment["text"].strip()
            f.write(f"{i + 1}\n{start_time} --> {end_time}\n{text}\n\n")
    return output_srt

def format_srt_time(seconds):
    """ Transformă secunde în formatul SRT hh:mm:ss,mmm """
    millisec = int((seconds % 1) * 1000)
    seconds = int(seconds)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02},{millisec:03}"

# 🔹 2. Funcție pentru adăugarea subtitrărilor pe video
def add_subtitles_moviepy(video_path, subtitle_path, output_video, font, color, fontsize):
    """ Adaugă subtitrările pe video folosind MoviePy """
    video = VideoFileClip(video_path)

    def generator(txt):
        from moviepy.video.VideoClip import TextClip
        return TextClip(txt, font=font, fontsize=fontsize, color=color, bg_color='black')

    subs = SubtitlesClip(subtitle_path, generator)
    video_with_subs = CompositeVideoClip([video, subs.set_position(("center", "bottom"))])
    video_with_subs.write_videofile(output_video, fps=video.fps)

# 🔹 3. Funcție principală
def process_video(video_path, font, color, fontsize):
    """ Extrage audio, transcrie textul și adaugă subtitrările """
    audio_path = "temp_audio.wav"
    ffmpeg_extract_audio(video_path, audio_path)
    
    model = whisper.load_model("small")
    whisper_result = model.transcribe(audio_path, language="en", word_timestamps=True)

    subtitle_file = save_subtitles(whisper_result)
    add_subtitles_moviepy(video_path, subtitle_file, OUTPUT_VIDEO, font, color, fontsize)

# 🔹 4. Creare interfață cu widget-uri
chooser = FileChooser('C:/Users/alex_/Downloads/')
chooser.filter_pattern = "*.mp4"

font_dropdown = widgets.Dropdown(
    options=['Arial', 'Courier', 'Times-New-Roman', 'Verdana'],
    value='Arial',
    description='Font:'
)

color_dropdown = widgets.Dropdown(
    options=['white', 'yellow', 'red', 'green', 'blue'],
    value='white',
    description='Culoare:'
)

fontsize_slider = widgets.IntSlider(
    value=30,
    min=10,
    max=60,
    step=2,
    description='Dimensiune:'
)

process_button = widgets.Button(description="Generează Subtitrări")
output_label = widgets.Label()

def on_button_click(b):
    if chooser.selected:
        output_label.value = "⏳ Procesăm video-ul..."
        process_video(chooser.selected, font_dropdown.value, color_dropdown.value, fontsize_slider.value)
        output_label.value = f"✅ Video salvat: {OUTPUT_VIDEO}"
    else:
        output_label.value = "⚠️ Selectează un fișier video mai întâi!"

process_button.on_click(on_button_click)

# Afișare widget-uri
display(chooser, font_dropdown, color_dropdown, fontsize_slider, process_button, output_label)


ModuleNotFoundError: No module named 'moviepy.editor'

In [3]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

In [3]:
import whisper
from moviepy.editor import VideoFileClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
from deep_translator import GoogleTranslator  # Bibliotecă pentru traducere
import os
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Setare pentru ImageMagick
os.environ["IMAGEMAGICK_BINARY"] = r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"

# 📍 Configurări inițiale
SUBTITLE_FILE = r"C:\Users\alex_\Downloads\subtitles.srt"
OUTPUT_VIDEO = r"C:\Users\alex_\Downloads\video_with_subtitles.mp4"

# 🔹 1. Funcție pentru generarea fișierului SRT cu timestamp-uri corecte
def save_subtitles(whisper_result, lang, output_srt=SUBTITLE_FILE):
    """ Salvează subtitrările într-un fișier .srt cu timestamp-uri reale și traduce dacă este necesar """
    translator = GoogleTranslator(source="en", target=lang) if lang != "en" else None

    with open(output_srt, "w", encoding="utf-8") as f:
        for i, segment in enumerate(whisper_result["segments"]):
            start_time = format_srt_time(segment["start"])
            end_time = format_srt_time(segment["end"])
            text = segment["text"].strip()
            
            if translator:
                text = translator.translate(text)  # Traducere text dacă este necesar
            
            f.write(f"{i + 1}\n{start_time} --> {end_time}\n{text}\n\n")
    
    return output_srt

def format_srt_time(seconds):
    """ Transformă secunde în formatul SRT hh:mm:ss,mmm """
    millisec = int((seconds % 1) * 1000)
    seconds = int(seconds)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02},{millisec:03}"

# 🔹 2. Funcție pentru adăugarea subtitrărilor pe video
def add_subtitles_moviepy(video_path, subtitle_path, output_video, font, color, fontsize):
    """ Adaugă subtitrările pe video folosind MoviePy """
    video = VideoFileClip(video_path)

    def generator(txt):
        from moviepy.video.VideoClip import TextClip
        return TextClip(txt, font=font, fontsize=fontsize, color=color, bg_color='black')

    subs = SubtitlesClip(subtitle_path, generator)
    video_with_subs = CompositeVideoClip([video, subs.set_position(("center", "bottom"))])
    video_with_subs.write_videofile(output_video, fps=video.fps)

# 🔹 3. Funcție principală
def process_video(video_path, font, color, fontsize, lang):
    """ Extrage audio, transcrie textul, traduce și adaugă subtitrările """
    audio_path = "temp_audio.wav"
    ffmpeg_extract_audio(video_path, audio_path)
    
    model = whisper.load_model("small")
    whisper_result = model.transcribe(audio_path, language="en", word_timestamps=True)

    subtitle_file = save_subtitles(whisper_result, lang)
    add_subtitles_moviepy(video_path, subtitle_file, OUTPUT_VIDEO, font, color, fontsize)

# 🔹 4. Creare interfață cu widget-uri
chooser = FileChooser('C:/Users/alex_/Downloads/')
chooser.filter_pattern = "*.mp4"

font_dropdown = widgets.Dropdown(
    options=['Arial', 'Courier', 'Times-New-Roman', 'Verdana'],
    value='Arial',
    description='Font:'
)

color_dropdown = widgets.Dropdown(
    options=['white', 'yellow', 'red', 'green', 'blue'],
    value='white',
    description='Culoare:'
)

fontsize_slider = widgets.IntSlider(
    value=30,
    min=10,
    max=60,
    step=2,
    description='Dimensiune:'
)

language_dropdown = widgets.Dropdown(
    options={'Engleză': 'en', 'Română': 'ro', 'Spaniolă': 'es', 'Franceză': 'fr', 'Germană': 'de', 'Italiană': 'it'},
    value='en',
    description='Limbă:'
)

process_button = widgets.Button(description="Generează Subtitrări")
output_label = widgets.Label()

def on_button_click(b):
    if chooser.selected:
        output_label.value = "⏳ Procesăm video-ul..."
        process_video(chooser.selected, font_dropdown.value, color_dropdown.value, fontsize_slider.value, language_dropdown.value)
        output_label.value = f"✅ Video salvat: {OUTPUT_VIDEO}"
    else:
        output_label.value = "⚠️ Selectează un fișier video mai întâi!"

process_button.on_click(on_button_click)

# Afișare widget-uri
display(chooser, font_dropdown, color_dropdown, fontsize_slider, language_dropdown, process_button, output_label)


FileChooser(path='C:\Users\alex_\Downloads', filename='', title='', show_hidden=False, select_desc='Select', c…

Dropdown(description='Font:', options=('Arial', 'Courier', 'Times-New-Roman', 'Verdana'), value='Arial')

Dropdown(description='Culoare:', options=('white', 'yellow', 'red', 'green', 'blue'), value='white')

IntSlider(value=30, description='Dimensiune:', max=60, min=10, step=2)

Dropdown(description='Limbă:', options={'Engleză': 'en', 'Română': 'ro', 'Spaniolă': 'es', 'Franceză': 'fr', '…

Button(description='Generează Subtitrări', style=ButtonStyle())

Label(value='')

In [3]:
from moviepy.editor import VideoFileClip, CompositeVideoClip
video = VideoFileClip(r"video_mpeg_still.avi")
print("Video format MPEG în care cadrele nu se modifică semnificativ")
video.ipython_display()

ModuleNotFoundError: No module named 'moviepy.editor'